In [2]:
import pandas as pd
from scipy import sparse
from implicit.als import AlternatingLeastSquares
import numpy as np

In [3]:
fileName = "/Users/rahulkhanna/Downloads/JWPlayer_takehome/jw_player_play_data.csv"

In [4]:
table = pd.read_csv(fileName, header=0)
table.columns

Index([u'Unnamed: 0', u'viewer_id', u'embed_id', u'media_id',
       u'publisher_category', u'play_seq', u'start_time', u'viewer_tz_offset',
       u'watched_duration', u'watched_pct', u'device_type', u'country_code',
       u'operating_system', u'browser', u'ad_starts', u'ad_skips',
       u'publisher'],
      dtype='object')

In [6]:
user_video = table[['viewer_id', 'media_id']]

user_video_counts = user_video.groupby(['viewer_id', 'media_id']).size().reset_index()
user_video_counts.columns = ['viewer_id', 'media_id', 'counts']
user_video_counts['viewer_id'] = user_video_counts['viewer_id'].astype("category")
user_video_counts['media_id'] = user_video_counts['media_id'].astype("category")

# user_video_counts.head(n=10)
user_video_counts[user_video_counts['viewer_id'] == 'd4614f09cddefb1ff996543c1b82fcbd65fdfdc9']

,viewer_id,media_id,counts
260249,d4614f09cddefb1ff996543c1b82fcbd65fdfdc9,2M8ZBJet,1
260250,d4614f09cddefb1ff996543c1b82fcbd65fdfdc9,2spw5dTi,1
260251,d4614f09cddefb1ff996543c1b82fcbd65fdfdc9,nTrJR9Bp,1
260252,d4614f09cddefb1ff996543c1b82fcbd65fdfdc9,rxG40GXY,1
260253,d4614f09cddefb1ff996543c1b82fcbd65fdfdc9,yN806lbc,1


In [10]:
videos = dict(enumerate(user_video_counts['media_id'].cat.categories))
video_ids = {v: i for i, v in videos.iteritems()}

users = dict(enumerate(user_video_counts['viewer_id'].cat.categories))
user_ids = {u: i for i, u in users.iteritems()}

print(len(users), len(videos))

(74190, 24955)


In [11]:
plays = sparse.csr_matrix((user_video_counts["counts"].astype("float"),
                           (user_video_counts["media_id"].cat.codes.copy(), 
                            user_video_counts["viewer_id"].cat.codes.copy())))

In [12]:
matrix_size = plays.shape[0]*plays.shape[1]
numOfWatches = len(plays.nonzero()[0])
sparsity = 100*(1 - (numOfWatches/float(matrix_size)))
sparsity

99.98512383619536

In [13]:
model = AlternatingLeastSquares(factors = 100, iterations = 50)

In [14]:
model.fit(40*plays)

In [15]:
[(videos[v], s) for v, s in model.similar_items(video_ids['nTrJR9Bp'])]


[('nTrJR9Bp', 1.0),
 ('1DY2Hdka', 0.48396072494263948),
 ('BwenFcGv', 0.40921574031283642),
 ('bTFism45', 0.36674815144575107),
 ('pZAejTqd', 0.36319319881539952),
 ('1Pf42Kxa', 0.35684397453987365),
 ('jvYY2xke', 0.35571016082335261),
 ('P11xeZEs', 0.35199833666004748),
 ('6irBuSpc', 0.3509114742995571),
 ('kq033H6s', 0.3493887997745746)]

In [17]:
user_1 = set(user_video_counts[user_video_counts['media_id'] == 'nTrJR9Bp']['viewer_id'])
user_2 = set(user_video_counts[user_video_counts['media_id'] == '1DY2Hdka']['viewer_id'])

len(user_1.intersection(user_2)), len(user_2)

(218, 223)

In [ ]:
count = 0
user_recommendations = {}
for key in user_ids:
    user_recommendations[key] = {}
    try:
        recs = model.recommend(userid=user_ids[key], user_items=40*plays, N=len(videos))
        for v, s in recs:
            user_recommendations[key][videos[v]] = s
    except:
        count+=1
        print key


91394f08d0173bd66a052b3a27b860ef67541b68
9cb5d29d7d5fe6d8a2f406bd7d0812a2bcb38d97
589901bbc92569207414d526165b44745d659724
5326c49ab00fdbe4f922c48cdaacce4d6bd55311
7267150d72f1e0c2da1c356fd58f0739b284f26d
5a0300c6222793d078237600a6efdcefc131a7d6
5aa540ecfe31494d09fdb77d9c3772e554d004ad
54cd5560ef351b0328d3b973b77c0cb1efa7e03f
5011183bb8fbc2ef993d44942707cfb5af80d88f
4ef08965d08e6144afa0f6acbf2f14be7e3ce5b1
91735c7527a552455adc9384761ace7c9e5624d9
5852274b99babade88284a66120fd2647db3ab15
58b757919f5711586b7591bc5171de8b8eae4020
7a69fc8b3c166798f203fb6ba204982c202824f1
a88b36f8ff5d3bbd476ff131113356309982fac9
da3e62cabe63b0fbe20a349ed3bd2c88c27d5e56
5bb9718ffc2faf27e06d5a285b5504dc0892e2f1
59f157beefdbf0ff988080735f48599d9f29ff5e
91385c7121356489291ccf7554bc6fbad6229899
7135229b8f1df3aee998a0bd0e5dea0929431315
5c1015d4f192ac16c564ba042f1f64444d7a7456
90de4c9a19569780c92904f65666ae731a40c251
4f33299fe3a44524b9d4a0428e34165f5e9fe1a6
6d041f607343b284bed67521b7833c8776c6f2a4
f35b28399ceebcfb

f02feaca8b6e2abc21d9734119105f9e9d78291b
6e0d7bfae8fdfd258e282512de918b4f72547f4f
53f48b347b90c0c6ca3f2ee732679051f1fe60c5
4fd9b837b040d5d210c1a053bbe70d72399a8ea6
53b019e43c48323ba187d3e02b2d92aeea20c431
b2256ddfdbb5d3f3bf1d11661b640bf91da3e1e2
4ff8a187fb1183c982380c33a6dc9326a8ebdffd
f6bcb764daaad450fbf42f68208ee054b9030de3
7995be82e2e57273b1d6e734104bce380cec3413
72375866419fce96aeee56bbcbeb06b760cb4ca9
4f2de97db92b18d221050889cd0fa61a42749023
6ef1c5b9b1f7aa036be9710757592dc8f0f7ebbe
71fd7096a65203d7a1086f143da8a1216ab090de
709861a4a13475b7634417d13faf39579fa8c618
7a12d2c950195450ba2bfede84d4f5606e18a6c7
edca0c55f86e94aeb09818d30857284e0dd21f3f
76d2094d3aca9ced4ca82ec77a8e99342330d11e
cf82705cd599164f5d322ed70a59f8b6d1ade24f
4fe702bad99756e51eadf6083ea52ceef680926d
5505827d9639da5f7fd99b35eeb94c8168c719bb
6d7080c6a8139335f5c54e75d5c8221cc8fadcb4
71f1994d09dd46bb7b837aa4cf9cf1fc6163ccfe
79565ed010c33f5c7a9aec26ab168a111f99a45d
5b8efc465e1a398cc81b80831fbd0d19472db906
50627992f60468e7

744674b681b21b8f3c2e53810e5145c19bd305bb
522f3677718f6f7b12df7e8c8d71b0367662dc18
741e5fbf580e389ef308badf7d117e3a0fddba7e
73eb247428aa7b07a0da885f219e75f61396803e
6e7c362454dbeef66005101431f6dfa0169da7ad
5b4b2e92d09d3b950be18e5662f653b55bb68bd1
7bf4eace1a15624a3c8c9878193300d0e1172bc2
fdd6656745c5f2d29dfa206e61b8e15ba2a908ff
5b1177ddb4f47b08b2ae3bd04401aedac068a173
50d3ba8b6d62fe54ea974b2b8fcabb68d37afb5d
9d8f186e228916a28156a7a667fc5f596ee1e7ae
5065d03188465f7341c8573fae808d47cdde6adb
5545f6689ebbb88cdb6986f2687d223d09c9e060
ecfd6c67ec55b16b70ef916e614495a96f1dd5bc
594dcffb5b8d07185e8b90c451b0517a699c3f3e
546594eced67621d5db05d2b9c7b979d7f5c81bb
526fc7c1be725100cf7c03a17198210b3d1fed87
b36b713312d375c18d2f0ca078fe5981cbafdfcf
541edef302f56c421e421cf3a67f297f83e01be2
83bf2820dc1b161423a253b96219dac11fb08ac2
7146a29b8a75669033899172d1a8af9835635e1d
4ec957a53fa0b58bc132cb1357723295d27584fd
d14b7b538862af12bea9ec338fc35bbafc39226f
57bd762903f4bf53964bfd3e1d8cb485ccd4a638
79d4e9cb57420c44

a80f4fce52e91c6c50a6510e6169473521a73f16
54bdf09eb87740a4472ec3ba8616920480bd820b
53c5833fd3f7c331a146d35466189e20e445bd4f
4f21763f46ddf54d1ee7f53563134563e996557b
54e1ef55b2743117a2f18fad486e61f048e89ed9
7305b67f3dc6fb0e54a3d721a665be6109ac3f35
bba91ea82ac7acfcee31dcb486dfe207b15a0dca
7a41d97b770fa2f21f16465e73f44b823b9090c2
707c43b10512567a0785f0c0205250f3a7dfd6c6
70e1afe344fe5d5924223c4580277b3a02d13c60
72c546a076d878fb52b90a6d75cc22b2ca7d340c
564c9912a0344265c2e3995a70e0335fa93ea3a7
6a992166d99953ab97b99aa909d16a07d48d0894
7bef0c660ae0752f27e7bc57b4d3b505b7896267
c74b14313b3748158f9979464551a40a34492279
51fc1c91b522166a12ba1695e678e8ae943a0f9b
541cadf1e6720558cd65d995a3384d134dd99810
56150f2b29bc43fbbdd95d52543ffb517d040b8f
f352d01d6272dcf377a2f94a36bb74c2981fe9d4
6ff1a5fc6b1dbab02208685e24a0fde107a05046
7193af158c3d7f3be4742d6d637584453bf770f8
58a541c0b5b705d48460804ef7fae5422b158c2f
65812499dfcb06a71621706c77f4befb6b8076b1
6f3acac2d21510e95c61289d4ae428de63baba37
585ba8fd229e01d6

534003ca3587093557862a64ba4d385917d2223b
4ed9805eb2319eceec02acc14d3009becce5b7ac
5c3f544b13b40cfe7523436333baf9a9b998b50d
791f6f76f1509a043a22e0090b4b5bf4aa02f527
73b83ce9bb324c15480b79a665bd42cf2e4801d0
5b5607aa037df6c564eeff219c30b5e701536008
68096ed595e59618da2ff7a1a51a2b87f03507f7
518751dfff57d6bc1c1ea5537323ed8894cc5999
6e1ff0db583381feb002594a9f6b36f711e883e8
6df61cc937e9771cc383a48117a04f2ce56b70b7
653037e1b8a89e6430b7a6d072975b0f0d7769b2
ac136bed29b6e08826e122619ebb48aab8e91a12
68d5c06a5a59bea702b0ddd6a1dce3e09816136e
5a95f1e1abc7fbe0ea3bdf070e3a9dc50cc27cc0
5486b31fa80229b2af215894ced4b008b6cf8752
71c01c22e2090f80c51bb0dc11e43c653e6a4d14
5572bc3cb2d6ffeae7a131814f87a57e3135810c
585f816c066d1fe288084f1ebbcb906df9f5f5d5
78c6912908451389d153b78455381f68cb233aa2
7922371986ba3f9dfbd0ee58d3ef7f68d5352dc3
794fa1fa46b619ce60d08688a6756239b917d862
5413209e0adbf94165e50957331d378b992dea27
51b1e0358b27a0971ef02587022d9182ad89af3a
53feff18219372f281d526116b9db6c1d0611ffb
c7d736e0753e6c63

...

b33f56bd734472c1a5b28261d253008b91d34262
82559cc8bc6c0ce14a218c5f4684a6a1f6124736
fc6ae6f5d022f3643f10aed2867a2378b91aff15
52d44cea492e72085fdd481816e425450b79e44a
795089a5a1a804c475eebb5c00c2fd1cc51da6cf
598c0e96845d153f140b02b2bfb685727ff2f6d8
79ab11f09fbdae72a1c5ed6ba8f9f32a8c16af69
63b031510e89b0750d4c55e296e6244d0950dba7
55475debec29730b4fdb85d1f097f6407ec5e7da
4fbd7896ef84c3aae1eaa92e7baae871d2e0f2d8
4fff395df505f36831f6d798d2b941febb509d7d
4eb780afd4c3dea41f5fbab266e6d5ef9475be37
567010d02de71366882c99f2675794fb6aff3770
592f4f802834947d0faa2ca2d5e054fafe70e5d2
5034ce51703c25fae2445757716c3e6b11856f76
6c73e3cdb6be3cb56abc443c86c1631cc796c7ad
7148d7d1c4d561eabaecc1dd9e625e9cd3a79dbf
6f893ecba23869556a398ae36e2f0439b111df7d
a5f2c599c18078b0a78e106fb9b5fc37da0603c6
56614a84514a7b367e3ecbe6fcb891df01187c29
7a049cc55bbdbcf06679b0ae7def01183c6fd59c
7898396e4800da7389ce7e2b8b617be8c37d3505
79dad4c0f7641833fd4c8b4800d56e704e4f4317
9096e19c180cd5bf17818acfd6d2dec676963e2e
4ef47e1170c6ac07

56249dfc1323120477697301ee264aa45ff658c2
6fa7ad2b92aa1a402adb4ac0883bb418c94eda93
565bb819e3c4bbc31f4b92b944971717d93403d1
85098ba1dd2d377cc643aa9ee3a4e9586744f6e2
593a1b7b4c261d9ad9e43216dd453b64ea2337dc
7927913a2bf66282d31f64fd74bee9f43c651351
6dd46f8d033d4c0782099b60e939a1a988dc0f03
505f6dd53db4f87db1d860ab8ca9613fdff3451c
521d36e9824a058b941a02e8c50904d752f2f1f2
6c6a470bbf64176821c210df777e308440e50392
56bc5cdb18221be17f7c06093cc7f94a633687ab
728838221ab8ce3902ad647e8c789e4818baaaa6
720167d9b5e132a312d8fa069b0ac9b3eef2478d
512898e94a0d5f6645d4050342d421891fef6711
78c4c9d4240d5887e193da87c41aeec395c15f7f
72031466f1418f8d2f62a7b2da6cc3f50ecfdd2d
c9ce8289a2c5b4d66b6488cc4fd20d16ac0edc4b
78993976daae0362746578ccd4867ca0d0bd3bb5
7a297cd0470cd4f415af7b6c0c61a24f1a95beae
77d6d3ae8e599334c289788b55d327503372452f
59b8cb9ac4b2527fc426b61633a3d2df526a4bd7
79a5b358d92965041b48705ca503a1bdc5c2633e
7a398aff1c010c0f0649fd230143d953b7c617db
6efe473f008820a59e735bb998131a7cecd2dd08
72cfbd61e1222e6c

5a5fa160f2de8b645f5b94700a9d3e240634e98a
557e62e4b22ecad18f0dc8bfabedc7ab423b8aad
75104341b020560494c816199777f6e1b499e6c9
6c77443e0752ad7897b82821ba06feb822a87e85
8cce42e4e55bf9ad107c1781ad2e54bdc5fbd10c
6f2a86a2df5994a1f9fadd49b7ecda1ddaf2d6ed
588d992475052be66aad6bcbb89069808ff785f7
7bebad1b35872de37285bbd055c4a93e5f53164c
5203679c2d108dc1b1cea98f6c860cc10c16fd24
6f5ca12ea631dfd3f47b14aad020a4aa16c2ac29
78c3a5df6c5ab297ce4df493a5fc4a4dae588b52
734ffb7a33c4aef0eade40ada8ffce6dc28af56b
a72f11f26ff75897076c3beb67e521b96e711e7c
6fd15372e0bfdc382f93ff2f581efb4fb661ee22
5c0f0ce482491fe6faff274eed3955c775a09ce3
725641bd635c4568343938cf082f097990e2ba79
b36eb689d99c1b585ea29fe0057884b09cf33205
57c48ec0cffe78414312b95763e3d48bbb5bcc14
5b583965f137297854902e0d57a149f28485d1d7
4f271faeafa35a38cd0f71a3b9492e3586202376
f14dd5e6ee9916851b689b073cd57abc59043b5f
7175216394fe3861fd5e203f0f4159c4e040ea47
8dbc999a3f656c6ce6f682d7ef0a8d9d21b181d5
70ab48e638ba09961279c2f55dd62ad1f97c7544
7ba562cd34b9c5a1

e28b4bd4cb686848a253bad02a477dc1fc3799ec
6a3de38873c3f546d2171daca5443c082ebfcd06
51aa7f249299ba6d10ed71853bc3e8461a6af58b
5ad9ca527bd29b73deff58a658e4b0d4c0efd0aa
6e600c0541e796f259247ee51f3f0e9c7af6e93e
6e7209889e164ddfa8d2816b657a31773b047fa4
869b2b0d17caed69168002d6f82278c0be270c4e
73c7fdc3b64ed4f213b58d18a6a4cc8a9cf9def6
aea24da932539ca7c23fbdcc3e40a939917da1e1
578af74635c420e07dd5f10571dbb0d7d088ce0d
7bb26801535a6475a01fc6a02fa3394b7ca0e6ee
55983e8e2f55b2bb4ff01ed8398161e13a4c5803
ea858526e0a970884773019321e1c930ebe95f77
5b6283790861ca9c47d427e30d8a747f0f97eaf1
5a5a995b62cccddbe85eed0b1e2e17171190f4d8
5ba242f581f029b0e0c4565b952a15c2ef41e0b1
f2581611081e3ce0d8e1ac8a2be5a1d454e41f77
6ff5a72f7df5f50179668dced203dea32691ea00
6f2d5d57703bed2b1be1363a1a5d741a96801188
6f5e53c8da137daa68179334427ba4695edde97f
59d4a3502f28837fe00f40763e121859b88409c0
5832fabdd715b0c4c44c163d6e0119c9a60a7fca
7336da25177da8e1fd5079b7015e76188135e1fc
555134c09f5de6a53e0472fb7c8321a09c5223a3
5783128ca42b89c9

6dc0d7e6f8af8c33be50246cb050eeae551707e4
56479f3bf6874c8d03dfe86ef4f18161d9f18be4
737ee4fff46124ec8da486dc85ecf36af3aebc03
592ca40a44221628824f8b8035f1f7ab6e8ba646
7c06145fd24b5d6a4eb4ab5342869735e7e1117b
6cf269a0ed9ceb27d504f272f61cf27e3866db78
4ff08387d6fbad30406ef77d56b2c6bd85c56c83
6db6cb0ad22b539e7d06c3a2b7184ef0e1c03df7
6b1aef4d8a4b5397e09ccdd6aa45c2b0c3b57047
574fe8b1d4c643b2b03b84557713a71d105fcd07
549b0bfb5b986de8c83fc5f0cd83d744574a0b05
797cb14705153ada416f198af0e980cc8286fae1
574855b771ca0c1ed0f79408993d955982725d86
72282157e078bd3c1c895e155987155007c99be7
4fd70f48ad45da83b9b86efc1517f3dcc6c3165c
54b1f516ddfd59273fb301d94c231413b5d7259c
56929e47a7a3151328e1b000cabe18338c9bc3c4
7955b013348c4868711e3f53393e3eea3a968243
5b5611dc664ccef050e52df1a1221a3f4ae43742
79f8b271e436344eeb86807590f1294738483dfb
58c7d03fa6e3f5d50fc41960a3b7479db0b5c7a0
586b6bb9423ef1576c0996883b2a1a94d85d54f0
6fbfc12921d3fb2b9fe99a1c69704ada55357072
4f609b9b5935bf92a49b46654076d6f0988bd035
512be5fc4272234f

In [282]:
recs = model.recommend(userid=user_ids["98c55737562526d54b5f7bd712902723ad6b5260"], 
                       user_items=40*plays)

IndexError: index (185) out of range

In [284]:
count

277

In [285]:
len(users)

330